In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../scraping/cleansed_data/players_20230515230306.csv', sep=';')

# Erstelle eine Kopie des Dataframes
df_copy = df.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18358 entries, 0 to 18357
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   LeagueCountry             18358 non-null  object 
 1   League                    18257 non-null  object 
 2   NationalLeagueLevel       18253 non-null  object 
 3   Club                      18352 non-null  object 
 4   No                        17950 non-null  float64
 5   Name                      18352 non-null  object 
 6   Value                     17890 non-null  float64
 7   ValueLastUpdate           17890 non-null  object 
 8   HighestValue              18000 non-null  float64
 9   HighestValueDate          18000 non-null  object 
 10  ValueCurrency             17890 non-null  object 
 11  Age                       18349 non-null  float64
 12  Height                    17069 non-null  float64
 13  Nationality               18352 non-null  object 
 14  Foot  

## Entfernen von Spalten und Zeilen

In [3]:
# Entferne Source, ScrapingTime, Instagram, Value-Dates & -Currency
df = df.drop(['Source', 'ScrapingTime', 'Instagram', 'ValueLastUpdate', 'HighestValueDate', 'ValueCurrency'], axis=1)

In [4]:
# Lösche alle Spieler ohne Value
df = df[df['Value'].notnull()]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17890 entries, 0 to 18357
Data columns (total 43 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   LeagueCountry             17890 non-null  object 
 1   League                    17890 non-null  object 
 2   NationalLeagueLevel       17890 non-null  object 
 3   Club                      17890 non-null  object 
 4   No                        17605 non-null  float64
 5   Name                      17890 non-null  object 
 6   Value                     17890 non-null  float64
 7   HighestValue              17890 non-null  float64
 8   Age                       17888 non-null  float64
 9   Height                    16811 non-null  float64
 10  Nationality               17890 non-null  object 
 11  Foot                      16205 non-null  object 
 12  Position                  17890 non-null  object 
 13  Consultancy               14606 non-null  object 
 14  Suppli

## Datentypen anpassen

In [6]:
# 'ClubSince' und 'ContractUntil' als Datumsangaben einlesen
df['ClubSince'] = pd.to_datetime(df['ClubSince'])
df['ContractUntil'] = pd.to_datetime(df['ContractUntil'])

In [7]:
# 'ClubSince' und 'ContractUntil' als float Werte umwandeln (Differenz in Tagen ab finalem Scrapingtag 17.05.2023, da dann auch die Values ermittelt wurden)
df['ClubSince'] = ((pd.to_datetime('2023-05-17') - df['ClubSince']) / np.timedelta64(1, 'D')).astype(float)
df['ContractUntil'] = ((df['ContractUntil'] - pd.to_datetime('2023-05-17')) / np.timedelta64(1, 'D')).astype(float)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17890 entries, 0 to 18357
Data columns (total 43 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   LeagueCountry             17890 non-null  object 
 1   League                    17890 non-null  object 
 2   NationalLeagueLevel       17890 non-null  object 
 3   Club                      17890 non-null  object 
 4   No                        17605 non-null  float64
 5   Name                      17890 non-null  object 
 6   Value                     17890 non-null  float64
 7   HighestValue              17890 non-null  float64
 8   Age                       17888 non-null  float64
 9   Height                    16811 non-null  float64
 10  Nationality               17890 non-null  object 
 11  Foot                      16205 non-null  object 
 12  Position                  17890 non-null  object 
 13  Consultancy               14606 non-null  object 
 14  Suppli

## Entferne Duplikate

In [9]:
# Doppelte Spieler aus dem Dataframe entfernen
df = df.drop_duplicates(subset=['Name', 'Age', 'Nationality', 'Club'], keep='first')

## Fehlende Werte ersetzen (Objekte und logische Werte)

In [10]:
# Erstelle ein Dictionary mit den Positionen und ihren entsprechenden Kategorien
pos_dict = {'Torwart': 'Torwart',
            'Innenverteidiger': 'Abwehr',
            'Linker Verteidiger': 'Abwehr',
            'Rechter Verteidiger': 'Abwehr',
            'Defensives Mittelfeld': 'Mittelfeld',
            'Zentrales Mittelfeld': 'Mittelfeld',
            'Linkes Mittelfeld': 'Mittelfeld',
            'Offensives Mittelfeld': 'Mittelfeld',
            'Linksaußen': 'Sturm',
            'Rechtsaußen': 'Sturm',
            'Mittelstürmer': 'Sturm',
            'Rechtes Mittelfeld': 'Mittelfeld',
            'Hängende Spitze': 'Mittelfeld',
            'Mittelfeld': 'Mittelfeld',
            'Abwehr': 'Abwehr',
            'Sturm': 'Sturm'}

# Erstelle eine neue Spalte mit der Positionskategorie
df['PositionCategory'] = df['Position'].map(pos_dict)
df['Cards'] = df['YellowCards'] + df['YellowRedCards'] + df['RedCards']

In [11]:
# Neue Spalte 'trending' erstellen
df['Trending'] = df.apply(lambda row: 1 if row['Value'] == row['HighestValue'] else 0, axis=1)

In [12]:
df.to_csv('../data/df_clean.csv')

## Nicht verwendeter Code

In [13]:
df['InternationalTeam'].fillna('kein_internationales_Team', inplace=True)

In [14]:
df.loc[df['InternationalTeam'] == 'kein_internationales_Team', ['InternationalGames', 'InternationalGoals']] = 0

In [15]:
# Setze die Torwart-spezifischen Werte auf 0, wenn die Position des Spielers keine Torwart-Position ist und der jeweilige Wert NaN ist
df.loc[(df['Position'] != 'Torwart') & (df['TwPenaltySavesQuote'].isna()), 'TwPenaltySavesQuote'] = 0
df.loc[(df['Position'] != 'Torwart') & (df['TwGoalsConceded'].isna()), 'TwGoalsConceded'] = 0
df.loc[(df['Position'] != 'Torwart') & (df['TwCleanSheets'].isna()), 'TwCleanSheets'] = 0

In [16]:
# Gruppieren Sie das DataFrame nach Position und berechnen Sie den Modus der Spieler-Nummern für jede Position
mode_no_by_position = df.groupby('Position')['No'].agg(lambda x: x.mode()[0])

# Funktion zum Imputieren von fehlenden Spieler-Nummern (No) basierend auf der Position
def impute_no_by_position(row):
    if pd.isnull(row['No']):
        return mode_no_by_position[row['Position']]
    else:
        return row['No']

# Anwenden der Funktion auf das gesamte DataFrame, um fehlende Spieler-Nummern zu imputieren
df['No'] = df.apply(impute_no_by_position, axis=1)

In [17]:
# Setze NaN-Werte in Karten-Werten auf 0
df['YellowCards'].fillna(0, inplace=True)
df['YellowRedCards'].fillna(0, inplace=True)
df['RedCards'].fillna(0, inplace=True)

In [18]:
# Objekte haben keine fehlenden Werte mehr
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17796 entries, 0 to 18357
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   LeagueCountry             17796 non-null  object 
 1   League                    17796 non-null  object 
 2   NationalLeagueLevel       17796 non-null  object 
 3   Club                      17796 non-null  object 
 4   No                        17796 non-null  float64
 5   Name                      17796 non-null  object 
 6   Value                     17796 non-null  float64
 7   HighestValue              17796 non-null  float64
 8   Age                       17794 non-null  float64
 9   Height                    16717 non-null  float64
 10  Nationality               17796 non-null  object 
 11  Foot                      16124 non-null  object 
 12  Position                  17796 non-null  object 
 13  Consultancy               14524 non-null  object 
 14  Suppli

In [19]:
# Fehlende Werte in Prozent pro Variable
print(df.isnull().sum() / len(df) * 100)

LeagueCountry                0.000000
League                       0.000000
NationalLeagueLevel          0.000000
Club                         0.000000
No                           0.000000
Name                         0.000000
Value                        0.000000
HighestValue                 0.000000
Age                          0.011238
Height                       6.063160
Nationality                  0.000000
Foot                         9.395370
Position                     0.000000
Consultancy                 18.386154
Supplier                    84.608901
ClubSince                    0.196673
ContractUntil                5.933918
Injury                       0.000000
InternationalTeam            0.000000
ActiveInternational          0.000000
FormerInternational          0.000000
InternationalGames           0.000000
InternationalGoals           0.000000
StartingElevenQuote         11.446392
MinutesQuote                11.446392
TwPenaltySavesQuote          1.612722
FsGoalPartic

In [20]:
# Manuelles Mapping erstellen
league_level_mapping = {
    "1.Liga": 1,
    "2.Liga": 2,
    "3.Liga": 3,
    "4.Liga": 4,
    "5.Liga": 5,
    "Reserveliga": 6,
    "Jugendliga": 6,
    "Playoff/Relegation": 7
}

# Ersetzen Sie die Werte in der Spalte "NationalLeagueLevel" basierend auf dem manuellen Mapping
df['NationalLeagueLevelCategory'] = df['NationalLeagueLevel'].map(league_level_mapping)

In [21]:
null_values = df['Minutes'].isnull().groupby([df['LeagueCountry'], df['League']]).sum().to_frame()
size_values = df.groupby(['LeagueCountry', 'League']).size()

null_values['Players'] = size_values
null_values = null_values.rename(columns={'Minutes': 'NullValues'})
null_values['NullValuesRelative'] = null_values['NullValues'] / null_values['Players'] * 100

null_values = null_values[null_values['Players'] > 20]  # exclude minor leagues
null_values = null_values.sort_values('NullValuesRelative', ascending=False)

print(null_values)

                                     NullValues  Players  NullValuesRelative
LeagueCountry League                                                        
Türkei        Süper Lig                     517      517          100.000000
Niederlande   Eredivisie                    483      483          100.000000
Griechenland  Super League 2                516      724           71.270718
Schottland    Championship                  155      239           64.853556
Norwegen      Eliteserien                   260      412           63.106796
Schweiz       Promotion League              247      392           63.010204
Spanien       LaLiga2                       315      533           59.099437
England       League One                    343      625           54.880000
Schweden      Allsvenskan                   196      389           50.385604
Tschechien    Fortuna Liga                  149      403           36.972705
Dänemark      Superligaen                    98      291           33.676976

In [22]:
# Ersetze NaN-Werte mit 'Unbekannt'
df['Nationality'].fillna('unbekannt', inplace=True)
df['Foot'].fillna('unbekannt', inplace=True)
df['Position'].fillna('unbekannt', inplace=True)
df['Supplier'].fillna('unbekannt', inplace=True)
df['Consultancy'].fillna('unbekannt', inplace=True)